In [1]:
import pandas as pd


In [11]:
f = open("amazon/Home_and_Kitchen.json", "r")
f2 = open("amazon/Home_and_Kitchen.json_10", "w")

num_lines_before = 0
num_lines_after = 0
for line in f:
    num_lines_before += 1
    if hash(line) % 10 == 0:
        f2.write(line + "\n")
        num_lines_after += 1

f.close()
f2.close()

print(num_lines_before)
print(num_lines_after)

21928568
2192549


In [ ]:
f = open("amazon/meta_Home_and_Kitchen.json")